In [1]:
#!pip install --user telepot

In [2]:
import telepot

In [3]:
token = '1033218796:AAEQh-5LC0EbJ8yIyMroTMNITtNREQJasHE'
bot = telepot.Bot(token)
print(bot.getMe())

{'id': 1033218796, 'is_bot': True, 'first_name': 'Gun Detection System', 'username': 'GunDetectionBot'}


In [4]:
import os, sys

import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage


# Root directory of the project
ROOT_DIR = os.path.abspath("Mask_RCNN")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log


from samples import gns

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

GNS_WEIGHTS_PATH = "Mask_RCNN/logs/gns20200104T2011/mask_rcnn_gns_0060_new.h5"  

Using TensorFlow backend.


In [5]:
def load_model():
    DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

    # Create model in inference mode
    #with tf.device(DEVICE):
    #    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,config=config)
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

    weights_path = GNS_WEIGHTS_PATH
    # Load weights
    model.load_weights(weights_path, by_name=True)
    model.keras_model._make_predict_function()

    return model

In [6]:
config = gns.gnsConfig()
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

GNS_DIR = "procdata"

# Load validation dataset
dataset = gns.gnsDataset()
dataset.load_gns(GNS_DIR, "train")

# Must call before using the dataset
dataset.prepare()

model = load_model()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [7]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [8]:
def load_image(image_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(image_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image

In [9]:
def gunRecognition(image_path, model=model):  
    predictedImage = image_path + '_prediction.png'
    isGun= False
    probability = ''
    
    image = load_image(image_path+'.png')

    # Run object detection
    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]

    if len(r['scores']) > 0:
        probability = r['scores']
        plt, image = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                        dataset.class_names, r['scores'], ax=ax,
                                        title="Predictions")
        for r in results:

            #plt.savefig(predictedImage)
            image = visualize.draw_box(image, r['rois'][0])
            skimage.io.imsave(predictedImage, image)
            isGun = True
        
    return predictedImage, isGun, probability
    

In [10]:
alert = u'\U0001F6A8'  

def handle(msg):
    try:
        content_type, chat_type, chat_id = telepot.glance(msg)
        if content_type == 'photo':
            image_path = "telegramImages/file"+str(chat_id)+"-"+str(msg['message_id'])
            bot.download_file(msg['photo'][-1]['file_id'], image_path+".png")
            retrievedData, isGun, probability = gunRecognition(image_path)     

            #retrievedData='gun detected'
            if isGun:
                retrieveText = alert+'ALERT!!!'+alert+'\nGun detected with {:.4f} confidence'.format(probability[0])
                #retrieveText = alert+'ALERT!!!'+alert+'\nGun detected with '+str(probability)+' confidence'
                bot.sendMessage(chat_id, text=retrieveText, parse_mode='markdown')
                bot.sendPhoto(chat_id, photo = open(retrievedData, 'rb'))
            else:
                retrieveText = 'No gun detected.'
                bot.sendMessage(chat_id, text=retrieveText, parse_mode='markdown')
    except Exception as e:
        bot.sendMessage(chat_id, text="Error: "+str(e))        

In [11]:
from telepot.loop import MessageLoop
MessageLoop(bot, handle).run_as_thread()

Processing 1 images
image                    shape: (719, 1280, 3)        min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max: 1280.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (422, 759, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (325, 600, 3)         min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1